# Import libraries
This notebook is built based on https://github.com/mginolin/standax/tree/main/notebooks from Madeleine Ginolin

In [1]:
import numpy as np
import pandas as pd
from standax import standardisation
from astropy.cosmology import Planck18
from scipy import stats

# Import sample

In [2]:
df = pd.read_parquet("../data/inference_data_frame.parquet")
new_cols = ["cov_mag_c", "cov_mag_x1", "cov_x1_c"]
df[new_cols] = 0.0
df = df.rename(columns={"magobs": "mag"})
df = df.rename(columns={"magobs_err": "mag_err"})
df = df.rename(columns={"prompt": "h_low"})
df = df.rename(columns={"localcolor": "localrestframe_gz"})
df = df.rename(columns={"localcolor_err": "localrestframe_gz_err"})

df["h_low"] = stats.norm.cdf(1, loc=df["localrestframe_gz"], scale=df["localrestframe_gz_err"])
df["h_low_err"] = 1e-4

In [3]:
print(df.head())

         mag   mag_err        x1    x1_err         c     c_err          h_low  \
0  19.137859  0.035667  0.153883  0.094231  0.072021  0.032548   6.911309e-01   
1  20.663308  0.035447  0.131785  0.048679 -0.038046  0.030231   1.000000e+00   
2  20.154988  0.036730  0.960991  0.202197 -0.068541  0.031945   2.871619e-42   
3  19.805727  0.036144 -0.892757  0.433325  0.007903  0.030300  1.539254e-104   
4  21.814985  0.034624 -0.040318  0.221775  0.299375  0.029686  3.887029e-195   

   localrestframe_gz  localrestframe_gz_err        z  cov_mag_c  cov_mag_x1  \
0           0.911980               0.176372  0.08755        0.0         0.0   
1           0.788434               0.022366  0.19305        0.0         0.0   
2           1.373041               0.027483  0.17005        0.0         0.0   
3           1.505770               0.023327  0.12095        0.0         0.0   
4           1.560177               0.018812  0.19435        0.0         0.0   

   cov_x1_c  h_low_err  
0       0.0  

In [4]:
data_to_fit = df[["mag", "mag_err", "x1", "x1_err", "c", "c_err", "h_low", "h_low_err",
                    "cov_mag_c", "cov_mag_x1", "cov_x1_c"]].dropna()

residuals = df['mag'] - Planck18.distmod(df['z']).value + 19.3
data_to_fit['mag'] = residuals

In [5]:
print(data_to_fit.head())

        mag   mag_err        x1    x1_err         c     c_err          h_low  \
0  0.357421  0.035667  0.153883  0.094231  0.072021  0.032548   6.911309e-01   
1  0.022311  0.035447  0.131785  0.048679 -0.038046  0.030231   1.000000e+00   
2 -0.181038  0.036730  0.960991  0.202197 -0.068541  0.031945   2.871619e-42   
3  0.275788  0.036144 -0.892757  0.433325  0.007903  0.030300  1.539254e-104   
4  1.157774  0.034624 -0.040318  0.221775  0.299375  0.029686  3.887029e-195   

   h_low_err  cov_mag_c  cov_mag_x1  cov_x1_c  
0     0.0001        0.0         0.0       0.0  
1     0.0001        0.0         0.0       0.0  
2     0.0001        0.0         0.0       0.0  
3     0.0001        0.0         0.0       0.0  
4     0.0001        0.0         0.0       0.0  


# Fit sample with total chi2

In [7]:
(best_params_l, sigmaint_l, mcmc_l), this_l = standardisation.standardise_snia(data_to_fit,
                                                      init=[-0.2, 3, 0.1], 
                                                      xkeys=["x1", "c", "h_low"], 
                                                      sigmaint_guess=0.15, 
                                                      model="linear", 
                                                      nfetch=3, verbose=True,
                                                     lmbda=1e4, fit_method="tncg")

fitting using <function fit_tncg at 0x128765ee0> using guess: {'offset': Array(0., dtype=float32), 'x_model': Array([[ 1.53883114e-01,  1.31785005e-01,  9.60990906e-01, ...,
         9.40427363e-01,  1.12440670e+00,  5.94145834e-01],
       [ 7.20211789e-02, -3.80455293e-02, -6.85407892e-02, ...,
        -2.62856260e-02, -1.20762875e-02,  4.03076142e-01],
       [ 6.91130877e-01,  1.00000000e+00,  2.87126055e-42, ...,
         9.98978078e-01,  3.23090035e-07,  9.98807788e-01]],      dtype=float32), 'coefs': Array([-0.2,  3. ,  0.1], dtype=float32)}
 -> fitting options {'lmbda': 10000.0, 'sigmaint': 0.15}
reaching sigmaint: 0.156746044754982
leading to parameters: {'coefs': Array([-0.14607033,  3.025537  ,  0.07123349], dtype=float32), 'offset': Array(-0.05917383, dtype=float32), 'x_model': Array([[ 1.4775534e-01,  1.3034269e-01,  9.2408270e-01, ...,
         9.5358032e-01,  1.1271503e+00,  5.9565955e-01],
       [ 8.7444082e-02, -2.6367452e-02, -4.9225684e-02, ...,
        -3.5958655e-

/Users/atrigui/miniconda3/envs/standax_no_metal/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
alpha = best_params_l['coefs'][0]
chain_alpha = mcmc_l.get_samples()['coefs'][:,0]
alpha_err = np.sqrt(np.sum((chain_alpha-alpha)**2)/len(chain_alpha))
beta = best_params_l['coefs'][1]
chain_beta = mcmc_l.get_samples()['coefs'][:,1]
beta_err = np.sqrt(np.sum((chain_beta-beta)**2)/len(chain_beta))
step = best_params_l['coefs'][2]
chain_step = mcmc_l.get_samples()['coefs'][:,2]
step_err = np.sqrt(np.sum((chain_step-step)**2)/len(chain_step))
offset = best_params_l['offset']
chain_offset = mcmc_l.get_samples()['offset']
offset_err = np.sqrt(np.sum((chain_offset-offset)**2)/len(chain_offset))

In [9]:
# Print parameters with errors in a clear table-like format
print(f"{'Parameter':<10} {'Value':>10} {'Error':>10}")
print(f"{'alpha':<10} {alpha:>10.4f} {alpha_err:>10.4f}")
print(f"{'beta':<10}  {beta:>10.4f} {beta_err:>10.4f}")

Parameter       Value      Error
alpha         -0.1461     0.0047
beta            3.0255     0.0255


In [10]:
samples = np.vstack([chain_alpha, chain_beta, chain_offset, chain_step]).T

## Save posterior

In [11]:
np.save("../data/results/standax_posterior.npy", samples)